<a href="https://colab.research.google.com/github/Ahsanm221/Mistral_7B_Fine-tuning/blob/main/Mistral_7B_version_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

# Define the file path
dataset_path = "/content/drive/MyDrive/datasets/dataset/alpaca_cleaned_dataset.json"

# Initialize empty lists for inputs and outputs
input_list = []
output_list = []

# Load the dataset and extract input/output pairs
with open(dataset_path, "r") as f:
    data = json.load(f)
    for entry in data:
        input_list.append(entry["input"])
        output_list.append(entry["output"])

# Print the number of entries to verify
print(f"Loaded {len(input_list)} inputs and {len(output_list)} outputs.")
# print(input_list[589])
# print(output_list[90])

Loaded 8740 inputs and 8740 outputs.


In [ ]:
instruction_string = """You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers.
Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details."""

# Initialize empty lists for formatted inputs and outputs
formatted_data = []

# Template with special tokens
example_template = lambda comment, response: f'''<s>[INST] {instruction_string} \n{comment} \n[/INST]\n{response}</s>'''

# Load the dataset, format each entry, and store in formatted_data list
with open(dataset_path, "r") as f:
    data = json.load(f)
    for entry in data:
        formatted_input = example_template(entry["input"], entry["output"])
        formatted_data.append(formatted_input)

# Print the number of formatted entries to verify
print(f"Formatted {len(formatted_data)} entries.")
print(formatted_data[100])

Formatted 8740 entries.
<s>[INST] You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers. 
Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details. 
{
    "Power": "0.0003094992",
    "Gain": "56.24548",
    "BW_3dB": "63244.55",
    "UGB": "438383300.0",
    "PM": "27.6918",
    "GM": "11.3984"
} 
[/INST]
Netlist:
M1 pch net3 VINP net2 VDD L1=258.0nm W1=67.0um
M2 pch net1 VINN net2 VDD L1=258.0nm W1=67.0um
M5 pch net2 VB1 VDD VDD L2=173.0nm W2=70.0um
M3 nch net3 net1 GND GND L3=207.0nm W3=47.0um
M4 nch net1 net1 GND GND L3=207.0nm W3=47.0um
M6 nch VOUT net3 GND GND L4=302

In [ ]:
import random

# Set the random seed for reproducibility
random.seed(42)

# Calculate the number of test samples (20% of the data)
test_size = int(len(formatted_data) * 0.2)

# Randomly sample indices for the test set
test_indices = random.sample(range(len(formatted_data)), test_size)

# Create the test set based on sampled indices
test_list = [formatted_data[i] for i in test_indices]

# Create the train set by excluding the test indices
train_list = [formatted_data[i] for i in range(len(formatted_data)) if i not in test_indices]

# Print the number of samples in each set to verify
print(f"Training examples: {len(train_list)}")
print(f"Testing examples: {len(test_list)}")
# print(train_list[0])
print(test_list[200])

Training examples: 6992
Testing examples: 1748


In [ ]:
print(test_list[200])

<s>[INST] You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers. 
Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details. 
{
    "Power": "4.118414e-05",
    "Gain": "60.45912",
    "BW_3dB": "26117.46",
    "UGB": "19841090.0",
    "PM": "73.4249",
    "GM": "37.4669"
} 
[/INST]
Netlist:
M1 pch net3 VINP net2 VDD L1=900.0nm W1=66.0um
M2 pch net1 VINN net2 VDD L1=900.0nm W1=66.0um
M5 pch net2 VB1 VDD VDD L2=772.0nm W2=6.0um
M3 nch net3 net1 GND GND L3=566.0nm W3=30.0um
M4 nch net1 net1 GND GND L3=566.0nm W3=30.0um
M6 nch VOUT net3 GND GND L4=424.0nm W4=97.0um
M7 pch VOUT

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset, DatasetDict

# Create a Hugging Face DatasetDict with train and test splits
data = DatasetDict({
    'train': Dataset.from_dict({"example": train_list}),
    'test': Dataset.from_dict({"example": test_list})
})

# Verify the dataset structure
print(data)
print(data['train'][0])
print(data['test'][0])

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 6992
    })
    test: Dataset({
        features: ['example'],
        num_rows: 1748
    })
})
{'example': '<s>[INST] You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers. \nYour task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.\nEnsure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details. \n{\n    "Power": "0.000568653",\n    "Gain": "10.73705",\n    "BW_3dB": "3383769.0",\n    "UGB": "16295760.0",\n    "PM": "0.0",\n    "GM": "0.0"\n} \n[/INST]\nNetlist:\nM1 pch net3 VINP net2 VDD L1=638.0nm W1=6.0um\nM2 pch net1 VINN net2 VDD L1=638.0nm W1=6.

In [ ]:
# Assuming `data` is your DatasetDict object
save_path = "/content/drive/MyDrive/datasets/MISTRAL7B_formatted_amplifier_data"

# Save the DatasetDict to the specified path
data.save_to_disk(save_path)

print(f"Dataset saved to {save_path}")

Saving the dataset (0/1 shards):   0%|          | 0/6992 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1748 [00:00<?, ? examples/s]

Dataset saved to /content/drive/MyDrive/datasets/MISTRAL7B_formatted_amplifier_data


In [ ]:
from datasets import load_from_disk

save_path = "/content/drive/MyDrive/datasets/MISTRAL7B_formatted_amplifier_data"
# Load the DatasetDict
loaded_data = load_from_disk(save_path)
print(loaded_data)

In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.7/422.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

In [ ]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4713: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
model.eval()  # Set model to evaluation mode (dropout modules are deactivated)


instructions_string = """You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers.
Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details."""
# Prompt template
prompt_template = lambda comment: f'''<s>[INST] {instructions_string} \n{comment} \n[/INST]'''

# Sample comment related to electronics for initial testing
comment = """Determine the resistor and capacitor values, transistor sizing, and biasing voltages for the provided two-stage amplifier based on the following metrics:
Power: 0.000568653, Gain: 10.73705, BW_3dB: 3383769.0, UGB: 16295760.0, PM: 0.0, GM: 0.0"""

# Generate the full prompt using the template
prompt = prompt_template(comment)
print(prompt)



<s>[INST] You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers. 
Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details. 
Determine the resistor and capacitor values, transistor sizing, and biasing voltages for the provided two-stage amplifier based on the following metrics:
Power: 0.000568653, Gain: 10.73705, BW_3dB: 3383769.0, UGB: 16295760.0, PM: 0.0, GM: 0.0 
[/INST]


In [ ]:
# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

# Decode and print output
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[INST] You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers. 
Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details. 
Determine the resistor and capacitor values, transistor sizing, and biasing voltages for the provided two-stage amplifier based on the following metrics:
Power: 0.000568653, Gain: 10.73705, BW_3dB: 3383769.0, UGB: 16295760.0, PM: 0.0, GM: 0.0 
[/INST] Based on the given amplifier performance metrics, I have designed a two-stage amplifier using TSMC 65nm technology. Here's the netlist format with the specified component values, transistor sizing, and bia

In [ ]:

model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=16,                       # Rank of the LoRA update matrix
    lora_alpha=16,             # Alpha scaling for LoRA layers
    target_modules=["q_proj"],  # Specify target modules for LoRA
    lora_dropout=0.02,          # Dropout for LoRA layers
    bias="none",               # No bias
    task_type="CAUSAL_LM"      # Causal language modeling task
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 266,604,544 || trainable%: 1.5732


In [ ]:
from datasets import load_from_disk

# Specify the path where the dataset is saved
dataset_path = "/content/drive/MyDrive/datasets/MISTRAL7B_formatted_amplifier_data"

# Load the dataset
dataset = load_from_disk(dataset_path)

# Verify the structure of the dataset
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 6992
    })
    test: Dataset({
        features: ['example'],
        num_rows: 1748
    })
})
{'example': '<s>[INST] You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers. \nYour task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.\nEnsure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details. \n{\n    "Power": "0.000568653",\n    "Gain": "10.73705",\n    "BW_3dB": "3383769.0",\n    "UGB": "16295760.0",\n    "PM": "0.0",\n    "GM": "0.0"\n} \n[/INST]\nNetlist:\nM1 pch net3 VINP net2 VDD L1=638.0nm W1=6.0um\nM2 pch net1 VINN net2 VDD L1=638.0nm W1=6.

In [ ]:
# Define a tokenize function
def tokenize_function(examples):
    # Extract the text field
    text = examples["example"]

    # Set truncation to the left, tokenize, and truncate to 1024 tokens max
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=1024  # Adjust based on your longest concatenated input-output strings
    )

    return tokenized_inputs

# Apply the tokenize function to your dataset
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/6992 [00:00<?, ? examples/s]

Map:   0%|          | 0/1748 [00:00<?, ? examples/s]

In [ ]:
# Define the pad token and data collator
tokenizer.pad_token = tokenizer.eos_token
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [ ]:
# Hyperparameters
lr = 1e-4                # Adjusted learning rate for stability with a larger dataset
batch_size = 8           # Increased batch size to optimize GPU usage
num_epochs = 3           # Reduced epochs to balance runtime and convergence

# Define training arguments
training_args = transformers.TrainingArguments(
    output_dir="mistral-ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="steps",
    logging_steps=50,                 # Log every 50 steps
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,    # For effective batch size and memory efficiency
    warmup_steps=100,                 # Increased warmup for smoother LR transition
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="wandb",                # Enable logging to W&B
    run_name="mistral_two_stage_amplifier_design" # Custom name for the W&B run
)

In [ ]:
# Configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)

# Train model
model.config.use_cache = False  # Disable cache warnings for training
trainer.train()

# Re-enable cache warnings after training
model.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss
0,0.354800,0.336752
2,0.276200,0.274993


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter sh

In [ ]:
from huggingface_hub import login
from google.colab import userdata
write_key = userdata.get('HFKey')  # paste your token here
login(write_key)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
hf_name = 'Ahsan221' # your hf username or org name
model_id = hf_name + "/" + "two_stage_amplifier_design"
model.push_to_hub(model_id)
trainer.push_to_hub(model_id)

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Ahsan221/mistral-ft/commit/61a2375e81a1ec515e84a21a57483d179230d6ac', commit_message='Ahsan221/two_stage_amplifier_design', commit_description='', oid='61a2375e81a1ec515e84a21a57483d179230d6ac', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel, PeftConfig

# # Set the model name from your Hugging Face Hub repository
# base_model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
# adapter_model_name = "ahsanm/two_stage_amplifier_design"  # Replace with your Hub model ID

# # Load the base model and tokenizer
# model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto", trust_remote_code=False)
# tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)

# # Load the fine-tuned LoRA adapter
# config = PeftConfig.from_pretrained(adapter_model_name)
# model = PeftModel.from_pretrained(model, adapter_model_name)

In [ ]:
# <s>[INST] You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers.
# Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
# Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details.
# {
#     "Power": "4.118414e-05",
#     "Gain": "60.45912",
#     "BW_3dB": "26117.46",
#     "UGB": "19841090.0",
#     "PM": "73.4249",
#     "GM": "37.4669"
# }
# [/INST]
# Netlist:
# M1 pch net3 VINP net2 VDD L1=900.0nm W1=66.0um
# M2 pch net1 VINN net2 VDD L1=900.0nm W1=66.0um
# M5 pch net2 VB1 VDD VDD L2=772.0nm W2=6.0um
# M3 nch net3 net1 GND GND L3=566.0nm W3=30.0um
# M4 nch net1 net1 GND GND L3=566.0nm W3=30.0um
# M6 nch VOUT net3 GND GND L4=424.0nm W4=97.0um
# M7 pch VOUT VB1 VDD VDD L5=217.0nm W5=4.0um
# Capacitor: C0 = 1.0pF
# Resistor: R0 = 3.0KOhm

# Biasing Voltages:
# VINP = 655.0
# VINN = 655.0
# VIN = 655.0
# VB1 = 764.0</s>

In [ ]:
instruction_string = """You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers.
Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details."""

prompt_template = lambda comment: f'''<s>[INST] {instruction_string} \n{comment} \n[/INST]'''

# Example comment
comment = '{\n    "Power": "4.118414e-05",\n    "Gain": "60.45912",\n    "BW_3dB": "26117.46",\n    "UGB": "19841090.0",\n    "PM": "73.4249",\n    "GM": "37.4669"\n}'

# Format the prompt
prompt = prompt_template(comment)
print(prompt)

<s>[INST] You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers. 
Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details. 
{
    "Power": "4.118414e-05",
    "Gain": "60.45912",
    "BW_3dB": "26117.46",
    "UGB": "19841090.0",
    "PM": "73.4249",
    "GM": "37.4669"
} 
[/INST]


In [ ]:
model.eval()

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=320)

# Decode and print output
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] You are a virtual electronics assistant specializing in analog design, specifically using TSMC 65nm technology for two-stage amplifiers. 
Your task is to determine the values of resistors, capacitors, MOSFET transistor sizing (i.e., lengths and widths for M1 to M5), and biasing voltages based on the provided amplifier performance metrics.
Ensure the response follows the netlist format and includes all specified component values (e.g., R0, C0, L1 to L5, W1 to W5), biasing voltages (VINP, VINN, VIN, VB1), and any relevant details. 
{
    "Power": "4.118414e-05",
    "Gain": "60.45912",
    "BW_3dB": "26117.46",
    "UGB": "19841090.0",
    "PM": "73.4249",
    "GM": "37.4669"
} 
[/INST]
Netlist:
M1 pch net3 VINP net2 VDD L1=902.0nm W1=82.0um
M2 pch net1 VINN net2 VDD L1=902.0nm W1=82.0um
M5 pch net2 VB1 VDD VDD L2=824.0nm W2=27.0um
M3 nch net3 net1 GND GND L3=700.0nm W3=19.0um
M4 nch net1 net1 GND GND L3=700.0nm W3=19.0um
M6 nch VOUT net3 GND GND L4=521.0nm W4=83.0um
M7 pch VOUT V

In [ ]:
# New comment example
comment = "Optimize the MOSFET transistor sizing for low power consumption."

# Re-format the prompt with new comment
prompt = prompt_template(comment)

# Run inference
model.eval()
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

# Decode and print output
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])